In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.insert(0, "../code-previous")

In [3]:
import pandas as pd
import numpy as np
import networkx as nx
import sklearn.feature_extraction as skfeatures
import utils
import time
import os

import seaborn as sns
sns.set_style('white')
import matplotlib.pylab as plt

import WOSutilities as wosutil

path2rawdata = '/mnt/disks/vault/wos2017-parsed/'

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
impact_df = pd.read_pickle("/mnt/disks/vault/impact_merge/impact_data_until_2010.pql")

In [5]:
# we need to load articles and categories

article_df = wosutil.load_wos_data(name = 'article', 
                                   path2rawdata = path2rawdata,
                                   year_list = None, 
                                   columns = ['ArticleID', 'PubYear', 'Doctypes', 'Title'], 
                                   dropna = ['ArticleID', 'PubYear'], 
                                   duplicate_subset = ['ArticleID'],
                                   isindict = {'Doctypes':np.sort(['Article','Letter','Review','Note'])}, 
                                   verbose = 50)

# select all categories for all the aricles published
category_list = np.array(['Physics, Applied'])  

category_df = wosutil.load_wos_data(name = 'category', 
                                    path2rawdata = path2rawdata,
                                    year_list = None,
                                    columns = None, 
                                    dropna = ['ArticleID'], 
                                    duplicate_subset = None,
                                    isindict = {'Label':category_list},
                                    verbose = 100)

50
100
150
200
250
300
350
400
450
Final DF Shape (39171592, 4)
100
200
300
400
Final DF Shape (1289055, 3)


In [6]:
# joining citations with articles and categories

# joining the dataframes
data = None
data = article_df.merge(category_df, 
                        how = 'left',
                        on = 'ArticleID')

data = data.merge(impact_df,
                  how = 'left',
                  on = 'ArticleID')

In [7]:
data.head()

,ArticleID,PubYear_x,Doctypes,Title,Type,Label,PubYear_y,c5
0,A1981LC59600003,1981,Article,SPORADIC LEGIONELLOSIS IN THE UNITED-STATES - ...,NaN,NaN,1981.0,48.0
1,A1981LC51800016,1981,Article,REACTION OF LEAD PERCHLORATE WITH AMMONIUM HYD...,NaN,NaN,1981.0,0.0
2,A1981MP52000011,1981,Article,ELECTRICAL PACING FOR SHORT BOWEL SYNDROME,NaN,NaN,1981.0,5.0
3,A1981LZ55900001,1981,Review,STABILITY TESTING OF SOLID DRUGS USING PREFERE...,NaN,NaN,1981.0,3.0
4,A1981ME20500010,1981,Article,ON THE DETERMINATION OF THE 1ST NORMAL STRESS ...,NaN,NaN,1981.0,1.0


In [8]:
data = data.drop_duplicates()
data.to_pickle("/mnt/disks/vault/citation-data-all/article-cat-c5-merged.pql")